In [51]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 160)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [64]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [4]:
rbn = db.routes_by_name()
route = rbn["220"].id
entries = db.snapshots(r=route, d=dt.date(2019, 3, 2))
stops_by_name = db.stops_by_name()
timetables = list(api.timetables("220", stops_by_name))
timetable_variants = {t for timetable in timetables for t in timetable.variants}

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [19]:
rbn = db.routes_by_name()
route = rbn["220"]
entries = db.snapshots(r=route.id, d=dt.date(2019, 3, 2))
stops_by_name = db.stops_by_name()
timetables = [t.value for t in db.timetables(route.id) if isinstance(t, util.Right)]
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [21]:
[type(list(t.variants)[0].stops[0]) for t in timetables]

[busboy.model.Stop,
 busboy.model.Stop,
 busboy.model.Stop,
 busboy.model.Stop,
 busboy.model.Stop]

In [22]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]), reverse=True)]

In [23]:
pvars = sorted(list(
    prediction.possible_variants(
        prediction.drop_duplicate_positions(entries_by_vehicle[vehicles_by_entry_count[0]]), 
        timetable_variants
    )),
    key = lambda t: t[0].poll_time)

In [24]:
order_pvars = list(prediction.check_variant_order(pvars))

In [65]:
route_sections = {v: list(prediction.route_sections(v.stops)) for v in timetable_variants}
stop_shaped_entries = [(e, {v: {t[1] for t in ts} for v, ts in util.dict_collect_set(vs, lambda tpl: tpl[0]).items()}) for (e, vs) in order_pvars]
times = prediction.stop_times(stop_shaped_entries, route_sections)
section_times = prediction.section_times(stop_shaped_entries, route_sections)
journeys = prediction.journeys(section_times)
journey_dfs = list(prediction.journeys_dataframe(prediction.estimate_arrival(times.items())))

In [66]:
longest = max(journey_dfs, key=lambda t: len(t[0].stops))
# longest[1] = longest[1].applymap(lambda e: e if e is None else (e[0].time().isoformat(), e[1].time().isoformat()))
# longest[1][['Ovens (Grange Road Terminus) [arrival]', 'Ovens (Grange Road Terminus) [departure]']] \
#     = pd.DataFrame(longest[1]['Ovens (Grange Road Terminus)'].tolist(), index=longest[1].index)
# longest[1][['Ovens (Grange Road Terminus) [arrival]', 'Ovens (Grange Road Terminus) [departure]']] \
#     = longest[1]['Ovens (Grange Road Terminus)'].apply(lambda c: [None, None] if c is None else list(c))
longest[1]

,Ovens (Grange Road Terminus) [arrival],Ovens (Grange Road Terminus) [departure],Ovens (Opp Grange Manor) [arrival],Ovens (Opp Grange Manor) [departure],Killumney Road (Kilumney Cross) [arrival],Killumney Road (Kilumney Cross) [departure],Ovens (EMC Terminus) [arrival],Ovens (EMC Terminus) [departure],Ballincollig West (Classes Lake) [arrival],Ballincollig West (Classes Lake) [departure],Ballincollig West (Opp Aylsbury Estate) [arrival],Ballincollig West (Opp Aylsbury Estate) [departure],Ballincollig West (Opp Old Quarry) [arrival],Ballincollig West (Opp Old Quarry) [departure],Ballincollig West (Opp Coolroe Heights) [arrival],Ballincollig West (Opp Coolroe Heights) [departure],Ballincollig West (Op White Horse Bar) [arrival],Ballincollig West (Op White Horse Bar) [departure],Ballincollig (Opp Jctn Barrys Road) [arrival],Ballincollig (Opp Jctn Barrys Road) [departure],Ballincollig (Shopping Centre) [arrival],Ballincollig (Shopping Centre) [departure],Ballincollig Town Centre (Garda St) [arrival],Ballincollig Town Centre (Garda St) [departure],Ballincollig (East Gate) [arrival],Ballincollig (East Gate) [departure],Ballincollig (Opp Rosewood Est) [arrival],Ballincollig (Opp Rosewood Est) [departure],Model Farm Rd (Guide Dog Centre) [arrival],Model Farm Rd (Guide Dog Centre) [departure],Model Farm Rd (Church Cross East) [arrival],Model Farm Rd (Church Cross East) [departure],Model Farm Rd (Inchaggin Eastbound) [arrival],Model Farm Rd (Inchaggin Eastbound) [departure],Model Farm Road (Eden Hall) [arrival],Model Farm Road (Eden Hall) [departure],Model Farm Rd (Opp Ultralase Ireland) [arrival],Model Farm Rd (Opp Ultralase Ireland) [departure],Model Farm Rd (IDA Technology Park) [arrival],Model Farm Rd (IDA Technology Park) [departure],Model Farm Rd (Dept of Agriculture) [arrival],Model Farm Rd (Dept of Agriculture) [departure],Model Farm Road (Farranlea Park) [arrival],Model Farm Road (Farranlea Park) [departure],Model Farm Rd (Bishopstown Park) [arrival],Model Farm Rd (Bishopstown Park) [departure],Model Farm Rd (Dennehy's Cross Jctn) [arrival],Model Farm Rd (Dennehy's Cross Jctn) [departure],Dennehy's Cross (Opp Cork Farm Ctr) [arrival],Dennehy's Cross (Opp Cork Farm Ctr) [departure],Victoria Cross (Victoria Lodge) [arrival],Victoria Cross (Victoria Lodge) [departure],Western Rd (Opp UCC Western Gateway) [arrival],Western Rd (Opp UCC Western Gateway) [departure],Western Rd (Opp UCC Castlewhite) [arrival],Western Rd (Opp UCC Castlewhite) [departure],Western Road (Gaol Crossl) [arrival],Western Road (Gaol Crossl) [departure],Western Road (Opp University College Gat [arrival],Western Road (Opp University College Gat [departure],Mardyke Walk (St. Joseph's School) [arrival],Mardyke Walk (St. Joseph's School) [departure],Mardyke (Presentation College) [arrival],Mardyke (Presentation College) [departure],Sheares Street (Mercy Hospital) [arrival],Sheares Street (Mercy Hospital) [departure],Grand Parade (Caseys Furniture) [arrival],Grand Parade (Caseys Furniture) [departure],South Mall (Opp Cork Passport Office) [arrival],South Mall (Opp Cork Passport Office) [departure],Cork City Hall [arrival],Cork City Hall [departure],Southern Rd (Opp Owl Printers) [arrival],Southern Rd (Opp Owl Printers) [departure],Douglas Road (Opp St Finbarrs Hospital) [arrival],Douglas Road (Opp St Finbarrs Hospital) [departure],Douglas Road (Glengesh Bellair) [arrival],Douglas Road (Glengesh Bellair) [departure],Douglas Road (Cross Douglas Rd Jctn) [arrival],Douglas Road (Cross Douglas Rd Jctn) [departure],Douglas Road (Before Woolharra Park) [arrival],Douglas Road (Before Woolharra Park) [departure],Douglas Road (Ardfallen Shopping Mall) [arrival],Douglas Road (Ardfallen Shopping Mall) [departure],Douglas Road (Endsleigh Estate) [arrival],Douglas Road (Endsleigh Estate) [departure],Douglas Road (Clermont Ave) [arrival],Douglas Road (Clermont Ave) [departure],Douglas East Village (Opp Tramway Tce) [arrival],Douglas East Village (Opp Tramway Tce) [departure],Maryb

In [53]:
to_time = lambda d: d.time().isoformat()
for variant, these_stop_times in times.items():
    print(f"Variant: {variant}")
    print(f"{len(these_stop_times)} journeys")
    for trip_number, trip_times in enumerate(these_stop_times):
        for stop_number, stop_time in enumerate(trip_times):
            to_time = lambda d: d.time().isoformat()
            try:
                stop_name = variant.stops[stop_number].name
            except IndexError:
                stop_name = "(IndexError)"
            print(f"- {trip_number}, {stop_number:2}, {stop_name:40}"
                  f" {stop_time.last_before.map(to_time).or_else(''):15}, {stop_time.first_after.map(to_time).or_else(''):15}")

Variant: (route: 201, start: Boherboy Rd (Opp Scoil Mhuire Banrion), end: CUH (Bishopstown Rd))
21 journeys
- 0,  0, Boherboy Rd (Opp Scoil Mhuire Banrion)                  , 07:19:32.536890
- 0,  1, Boherboy Road (Lotabeg Estate)           07:18:52.525808, 07:20:32.577308
- 0,  2, North Ring Road (Opp Mayfield Supermarke 07:20:12.559419, 07:21:32.628519
- 0,  3, North Ring Road (Lagan Grove)            07:21:32.628519, 07:22:52.652098
- 0,  4, North Ring Rd (Glencree Crescent)        07:22:52.652098, 07:23:32.658942
- 0,  5, North Ring Rd (Corrib Lawn)              07:23:32.658942, 07:24:32.690832
- 0,  6, North Ring Rd (Boyne Crescent)           07:24:32.690832, 07:25:12.698864
- 0,  7, North Ring Rd (Opp Riverview Estate)     07:25:52.734574, 07:27:32.789463
- 0,  8, Old Commons Rd (Opp Topaz Service St)    07:29:32.858933, 07:30:52.904094
- 0,  9, Farranferris Ave (Pophams Rd Junction)   07:30:52.904094, 07:32:32.993772
- 0, 10, Pophams Rd (Opposite Community Centre)   07:31:52.958

In [50]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [51]:
notebook.show_timetables(themap, timetables)

In [56]:
themap.map.add_control(lf.LayersControl())

In [52]:
[(v, len(entries_by_vehicle[v])) for v in vehicles_by_entry_count]

[(VehicleId(raw='7338674957838189376'), 7885), (VehicleId(raw=None), 658)]